In [1]:
! python -m pip install google-genai
! python -m pip install pypdf

  Using cached google_genai-1.60.0-py3-none-any.whl.metadata (53 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached websockets-15.0.1-cp313-cp313-win_amd64.whl.metadata (7.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.5-cp313-cp313-win_amd64.whl.metadata (7.4 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached pyasn1-0.6.2-py3-none-any.whl.metadata (8.4 kB)
Using cached google_genai-1.60.0-py3-none-any.whl (719 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
Using cached pydantic_core-2.41.5-cp313-cp313-win_amd64.whl (2.0 MB)
Using cac


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


8. Skapa en chattbot som svarar på frågor utifrån något dokument som du själv väljer. 

In [ ]:
from google import genai
from pypdf import PdfReader
api_key = '---'
client = genai.Client(api_key=api_key)

Läser in PDF-filen som är en gammal rapport som jag gjorde i förra AI-kursen.

In [2]:
reader = PdfReader("pdf/Kunskapskontroll_rapport_GencGashi.pdf")

text = ""
for page in reader.pages:
    text += page.extract_text()

Chunks

Delar in texten i chunks så att modellen hanterar den enklare.

In [3]:
chunks = []
n = 1000
overlap = 200

for i in range(0, len(text), n - overlap):
    chunks.append(text[i:i + n])

print(f"Antal chunks: {len(chunks)}.")

Antal chunks: 25.


Jag skapar embeddings för representera texten som numeriska vektorer.

In [4]:
from google.genai import types

def create_embeddings(text):
    model = "gemini-embedding-001"
    task_type = "SEMANTIC_SIMILARITY"
    return client.models.embed_content(
        model=model,
        contents=text,
        config=types.EmbedContentConfig(task_type=task_type)
    )

embeddings = create_embeddings(chunks)

Semantic Search

In [5]:
import numpy as np

def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1)) * np.linalg.norm(vec2)

Gör en sökning igenom textchunks och returnerar mest relevanta utifrån likheten mellan query och chunk-embeddings.

In [6]:
def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = create_embeddings(query).embeddings[0].values
    similarity_scores = []

    for i, chunk_embedding in enumerate(embeddings.embeddings):
        similarity_score = cosine_similarity(
            query_embedding,
            chunk_embedding.values
        )
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]

    return [chunks[index] for index in top_indices]


Söker efter text som är mest relevant till frågan.

In [7]:
fråga = "Vilken dataset användes?"
svar = semantic_search(fråga, chunks=chunks, embeddings=embeddings, k=1)
print(svar)

['tienters sjukvårdskostnader baserat på BMI, \nkön, region, ålder, rökstatus och antal barn. \nDetta dataset valdes eftersom den passar ett regressions problem mycket väl. Den kändes lagom \nutmanande då den inte är extremt stor, vilket gör den hanterbar. Samtidigt finns det vissa \nsvårigheter, till exempel att encoda de kategoriska variablerna.  En risk med datasetet är att det är \nrelativt litet och om man rensar extremvärden kan det påverka resultaten så att prediktionerna blir \nmindre korrekta. \n \n3.2 Tillgång till data \n \nNästa steg var att läsa in CSV-filen i Jupyter Notebook och därefter hämta information om datasetet, \ndess beskrivning samt de första raderna. Därefter delades datan upp i tränings och testdata enligt \nfigur 4. 5 \n \n \nFigur 4: Uppdelning av data. \nTräningsdatan användes i de kommande stegen, medan testdatan sparades för att senare utvärdera \nden bästa modellen.  \n \n3.3 EDA analys & Databearbetning \n \nEn EDA-analys utfördes på träningsdatan där 

Generera svar

Skapar en prompt som besvarar på frågor baserat på PDF:en och säger att den inte vet om informationen saknas.

In [8]:
system_prompt = """Jag kommer ställa dig en fråga, och jag vill att du svarar
baserat bara på kontexten jag skickar med, och ingen annan information.
Om det inte finns nog med information i kontexten för att svara på frågan,
säg "Det vet jag inte". Försök inte att gissa.
Formulera dig enkelt och dela upp svaret i fina stycken. """

Här skapar jag en användarprompt som kombinerar frågan med relevant information från texten.

In [9]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks, embeddings))
    user_prompt = f"Frågan är {query}. Här är kontexten: {context}."
    return user_prompt

Här skickas en systemprompt och en användares fråga till modellen som sedan returnerar ett svar.

In [10]:
def generate_response(system_prompt, user_message, model="gemini-2.0-flash"):
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt),
            contents=generate_user_prompt(user_message)
        )
    return response

In [11]:
print(generate_response(system_prompt, "Vad är RMSE?").text)

RMSE står för Root Mean Square Error.

Det visar hur bra en regressionsmodell predikterar nya värden.

Det beräknas genom att ta kvadratroten ur medelvärdet av de kvadrerade skillnaderna mellan de verkliga och predikterade värdena.



In [12]:
print(generate_response(system_prompt, "Vad är RAG?").text)

Det vet jag inte.


Evaluering - Skriver en fråga och det förväntade svaret.

In [13]:
validation_data = [
{"question": "Vilka delar utgör en RAG-modell?",
"ideal_answer": """En RAG-modell innehåller två delar: 
en retriever som söker efter relevanta stycken i en text, 
och en generator som genererar svar utifrån den givna kontexten."""}
]

print(validation_data[0]["question"])
print()
print(validation_data[0]["ideal_answer"])

Vilka delar utgör en RAG-modell?

En RAG-modell innehåller två delar: 
en retriever som söker efter relevanta stycken i en text, 
och en generator som genererar svar utifrån den givna kontexten.


Det här är en system-prompt som fungerar som ett utvärderingssystem. Den ger oss poäng (1, 0.5 eller 0) baserat på hur bra AI-modellens svar matchar det önskade svaret.

In [14]:
evaluation_system_prompt = """Du är ett intelligent utvärderingssystem vars uppgift är att utvärdera en AI-assistents svar. 
Om svaret är väldigt nära det önskade svaret, sätt poängen 1. Om svaret är felaktigt eller inte bra nog, sätt poängen 0.
Om svaret är delvis i linje med det önskade svaret, sätt poängen 0.5. Motivera kort varför du sätter den poäng du gör.
"""

Här skickas frågan till modellen, genererar ett svar, och skickar sedan både AI-svaret och det önskade svaret till modellen för utvärdering, och skriver sedan ut utvärderingsresultatet.

In [15]:
query = validation_data[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data[0]['ideal_answer']}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 0
Motivering: Assistenten svarar "Jag vet inte" vilket inte är ett användbart svar.


In [16]:
validation_data_2 = [
{"question": "Vilket dataset har använts?",
"ideal_answer": "Medical Cost Personal"}
]
print(validation_data_2[0]["question"])
print(validation_data_2[0]["ideal_answer"])

Vilket dataset har använts?
Medical Cost Personal


In [17]:
query = validation_data_2[0]["question"]

response = generate_response(system_prompt, query)
print(response.text)

Datasetet som har använts är Medical Cost Personal.
Det består av 1 338 rader och 7 kolumner:

*   Age – patientens ålder.
*   Sex – patientens kön (Male/Female)
*   BMI - Body Mass Index.
*   Smoker – rökstatus (Yes/No)
*   Children – antal barn som patienten har.
*   Region – patientens boenderegion.
*   Charges – patientens sjukvårdskostnader.


In [18]:
evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data_2[0]['ideal_answer']}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 1
Motivering: AI-assistenten ger ett korrekt svar baserat på kontexten.


In [19]:
validation_data_3 = [
{"question": "Vilka modeller inom machine learning har använts?",
"ideal_answer": "Linear Regression, Decision Tree och Random Forest"}
]
print(validation_data_3[0]["question"])
print(validation_data_3[0]["ideal_answer"])

Vilka modeller inom machine learning har använts?
Linear Regression, Decision Tree och Random Forest


In [20]:
query = validation_data_3[0]["question"]

response = generate_response(system_prompt, query)
print(response.text)

Följande modeller inom machine learning har använts:

*   Linjär regression.
*   Random Forest.
*   Decision Tree.


In [21]:
evaluation_prompt = f"""Fråga: {query}
AI-assistentens svar: {response.text}
Önskat svar: {validation_data_3[0]['ideal_answer']}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)

Poäng: 1
Motivering: Svaret innehåller samtliga modeller som har använts.


En enkel applikation som är en chattbot som kan svara på frågor om min kunskapskontroll från förra kursen.

In [22]:
print("Chatbot")

while True:
    fråga = input("")
    if fråga.lower() == "q":
        break
    
    print("\nFråga:", fråga)
    svar = generate_response(system_prompt, fråga)
    print("\nAI-svar:", svar.text, "\n")


Chatbot

Fråga: vilket dataset användes?

AI-svar: Datasetet som användes var "Medical Cost Personal". 


Fråga: berätta om rymden

AI-svar: Det vet jag inte. 


Fråga: Vad är RMSE?

AI-svar: RMSE visar hur bra en regressionsmodell predikterar nya värden.

Det beräknas genom att ta kvadratroten ur medelvärdet av de kvadrerade skillnaderna mellan de verkliga och predikterade värdena.

Formeln för RMSE är:

`𝑅𝑀𝑆𝐸 = √1/𝑛 ∑ (𝑦𝑖  −  𝑦̂1)^2𝑛
𝑖=0`

Här är:
*   𝑦𝑖 de verkliga värdena,
*   𝑦̂1 motsvarar de predikterade värdena
*   n är antalet observationer. 


Fråga: Vad är CNN?

AI-svar: Det vet jag inte. 

